### Initialization and general setup
This file is automatically generated using equi2py. Created on 2023-11-29 10:33:23



In [3]:
import os
from pathlib import Path
import numpy as np
from collections import OrderedDict

import chemapp as ca
from chemapp.friendly import (
    Info,
    StreamCalculation as casc,
    ThermochemicalSystem as cats,
    EquilibriumCalculation as caec,
    Units as cau,
    TemperatureUnit,
    PressureUnit,
    VolumeUnit,
    AmountUnit,
    EnergyUnit,
    Status,
)
from chemapp.core import (
    LimitVariable,
    ChemAppError,
)

This program contains ChemApp
Copyright GTT-Technologies, Kaiserstrasse 103, D-52134Herzogenrath, Germany
http://www.gtt-technologies.de

By using the ChemApp library, You accept and agree to be bound by the terms 
and conditions of its license, as specified in the file LICENSE. This is a 
human-readable summary of (and not a substitute for) the license:

 - Free of charge for FactSage users in active M&S until 31st August 2024.
 - Internal use of the library and derivative works only.
 - No reverse-engineering of software or data.
 - No result databases or derivative works based on them may be shared with
   third parties without consent by GTT. 



### Load data file


In [4]:
cst_file = Path("HotMetalLiquid.cst")
# load the database file
cats.load(cst_file)

# set the limits for target calculations, if deviating from the standards as below:
# T = 250.0 - 10000.0 K
# P = 1e-35 - 1e08 bar
# V = 1e-08 - 1e35 dm3
# note that these limits are always set in the above units
ca.basic.tqclim(LimitVariable.TLOW, 100.0)

### Units setup
It is possible to change the units before retrieving results, or in between making multiple inputs. However, mixed

unit input is currently not generated automatically.



In [5]:
cau.set(
    T=TemperatureUnit.C,
    P=PressureUnit.atm,
    V=VolumeUnit.dm3,
    A=AmountUnit.tonne,
    E=EnergyUnit.J,
)

In [6]:
import pandas as pd

# reading the csv file, setting the index to the index used in the csv file.
# remember to configure and convert units, if needed.
input_data = pd.read_csv("hotmetal.csv").set_index("ID")

### Iterating over the table entries


In [7]:
# setting general boundary conditions of the calculation(s)
casc.set_eq_P(1.0)
casc.create_st(name="#1", T=1300, P=1)
casc.create_st(name="#2", T=25, P=1)
A = 0.0
A_range = [0.0]


# we keep the results in an OrderedDict, to access by variable value
# if wanted, the .values() attribute of results can be easily used like a list.
results = OrderedDict()

# iteration over the rows - since ID is set as index

for ID, row_data in input_data.iterrows():
    # the order and association of the table values is inferred (csv file header)
    total_mass, T, Fe, C, Si, Mn, P, S = row_data
    for A in A_range:
        # for each row, temperature was different
        casc.set_eq_T(T)

        # set incoming amounts
        # since this is a calculation with initial conditions, input needs to be made
        # using phase constituents

        casc.set_IA_pc("#1", "Fe_bcc(s)", "Fe_bcc(s)", Fe)
        casc.set_IA_pc("#1", "C_Graphite(s)", "C_Graphite(s)", C)
        casc.set_IA_pc("#1", "Si_solid(s)", "Si_solid(s)", Si)
        casc.set_IA_pc("#1", "Mn_Solid_Alpha(s)", "Mn_Solid_Alpha(s)", Mn)
        casc.set_IA_pc("#1", "P_Solid_(white)(s)", "P_Solid_(white)(s)", P)
        casc.set_IA_pc("#1", "S_alpha_orthorhombic_(s)", "S_alpha_orthorhombic_(s)", S)

        # the variable 'A' is the amount of oxygen, which is still fixed at 0.0
        casc.set_IA_pc("#2", "gas_ideal", "O2", A)

        try:
            # calculate
            casc.calculate_eq()
            # fetch results
            results[(ID, A)] = casc.get_result_object()
        except ChemAppError as c_err:
            print(f"[<A> = { A }] ChemApp Error {c_err.errno}:")
            print(c_err)
            # change this if you want to capture a calculation error
            # maybe input a dummy value (or other, adequate replacements?):
            # results[A] = None
            pass

#### Generate the hot liquid metal streams

In [8]:
# we want to 'pick' the Fe-liq phase status from the result objects, and use it
# as input to following calculations. Using `create_stream` yields 'StreamState'
# classes, which are exactly for that usecase.
iron_liquid_streams = OrderedDict()
for combined_ID, calculation_result in results.items():
    table_ID, amount_O2 = combined_ID
    iron_liquid_streams[table_ID] = calculation_result.create_stream(
        name="Iron Liquid", phs_include=["Fe-liq"]
    )

In [9]:
# Inspecting the StreamState object does not give different numbers than the
# full equilibrium calculation, because the Fe-liq phase is the only stable
# phase.
print(iron_liquid_streams[2])

Stream State
---------------------------------------------------------------
Class:       chemapp.core.StreamState

Name:        Iron Liquid
T:           1.3578E+03 C
P:           1.0000E+00 atm
Cp:          7.9641E+07 J/C
H:           1.2601E+11 J
S:           1.8816E+08 J/C
G:           -1.8087E+11 J
V:           0.0000E+00 dm3
---------------------------------------------------------------
Phase                     Constituent                         A
                                                          tonne
---------------------------------------------------------------
Fe-liq                    Fe                         9.5020E+01
Fe-liq                    C                          3.9500E+00
Fe-liq                    Mn                         4.0000E-01
Fe-liq                    P                          1.0100E-01
Fe-liq                    S                          1.9000E-02
Fe-liq                    Si                         5.1000E-01
----------------------------

### Calculate using the input streams, varying A (amount of oxygen)



In [10]:
cst_file = Path("LDprocess.cst")
# load the database file
cats.load(cst_file)

# set the limits for target calculations, if deviating from the standards as below:
# T = 250.0 - 10000.0 K
# P = 1e-35 - 1e08 bar
# V = 1e-08 - 1e35 dm3
# note that these limits are always set in the above units
ca.basic.tqclim(LimitVariable.TLOW, 100.0)

cau.set(
    T=TemperatureUnit.C,
    P=PressureUnit.atm,
    V=VolumeUnit.dm3,
    A=AmountUnit.tonne,
    E=EnergyUnit.J,
)

# setting general boundary conditions of the calculation(s)
casc.set_eq_T(1650)
casc.set_eq_P(1.0)

casc.create_st(name="#2", T=25, P=1)
casc.create_st(name="#1", T=1300, P=1)
A = 0.0

# original condition for A was 10.0, which would not include 10.0.
# Increasing it by 0.2 yields the inclusive upper boundary.
A_range = np.arange(0.0, 10.2, 0.2)

# we keep the results in an OrderedDict, to access by variable value
# if wanted, the .values() attribute of results can be easily used like a list.
from collections import defaultdict

converter_results = defaultdict(dict)

for ID, stream_object in iron_liquid_streams.items():
    for A in A_range:
        # set incoming amounts
        # since this is a calculation with initial conditions, input needs to be made
        # using phase constituents
        casc.set_st(stream_object, A=input_data.loc[ID, "mass"])
        casc.set_IA_pc("#2", "gas_ideal", "O2", A)
        try:
            # calculate
            casc.calculate_eq()
            # fetch results
            converter_results[ID][A] = casc.get_result_object()
        except ChemAppError as c_err:
            print(f"[<A> = { A }] ChemApp Error {c_err.errno}:")
            print(c_err)
            # change this if you want to capture a calculation error
            # maybe input a dummy value (or other, adequate replacements?):
            # results[A] = None
            pass

In [13]:
# inspecting the results - now this is a nested dictionary, so as an example,
# only one of them is checked
print("all IDs calculated", converter_results.keys())
print("tons of oxygen", converter_results[2].keys())

all IDs calculated dict_keys([0, 1, 2, 3, 4])
tons of oxygen dict_keys([0.0, 0.2, 0.4, 0.6000000000000001, 0.8, 1.0, 1.2000000000000002, 1.4000000000000001, 1.6, 1.8, 2.0, 2.2, 2.4000000000000004, 2.6, 2.8000000000000003, 3.0, 3.2, 3.4000000000000004, 3.6, 3.8000000000000003, 4.0, 4.2, 4.4, 4.6000000000000005, 4.800000000000001, 5.0, 5.2, 5.4, 5.6000000000000005, 5.800000000000001, 6.0, 6.2, 6.4, 6.6000000000000005, 6.800000000000001, 7.0, 7.2, 7.4, 7.6000000000000005, 7.800000000000001, 8.0, 8.200000000000001, 8.4, 8.6, 8.8, 9.0, 9.200000000000001, 9.4, 9.600000000000001, 9.8, 10.0])


#### Figure out which amount of oxygen yields < 0.1wt% carbon

This is just one simple way of 'processing' the data. Computationally, saving
all the previous calculations and then only checking for this does not make a
lot of sense - it would be much easier and removing unnecessary calculations if
we ask for the relevant properties directly after calculating (and without
fetching the result object). Since we know that the solution is monotoneous, a
different search strategy for the oxygen amount (e.g. binary search...), or even
a black-box optimization routine would yield better and quicker results.

In [16]:
# adapt this value to your needs
target_wp_c = 0.1

# since the input_data field has only been read, a copy is not strictly needed
output_table = input_data.copy(deep=True)

# we create a DataFrame with columns for the values we want to operate on
carbon_content_in_Fe_liq = pd.DataFrame(
    columns=["Amount oxygen", "Amount Fe-liq", "Carbon content", "wt% C", "dH"],
    dtype=float,
).set_index("Amount oxygen")

for ID in input_data.index:
    for amount_O2, calc_res in converter_results[ID].items():
        # the total phase amount
        amount_feliq = calc_res.phs["Fe-liq"].A

        dH = calc_res.dH
        # the phase constituent amount
        amount_c = calc_res.phs["Fe-liq"].scs["C"].A
        # calculate carbon content
        wp_c = amount_c / amount_feliq * 100
        # set the row
        carbon_content_in_Fe_liq.loc[amount_O2, :] = amount_feliq, amount_c, wp_c, dH
        # set the target wp C

    # relatively crude search method through the results
    for A, wp_C in carbon_content_in_Fe_liq["wt% C"].items():
        if wp_C < target_wp_c:
            min_o2_to_add = A
            min_o2_dH = carbon_content_in_Fe_liq["dH"].loc[A]
            break
    print(
        f"you have to add at least {min_o2_to_add:.2f}t so that the carbon content of Fe-Liq drops below 0.1 w% C"
    )

    # write the values to the respective places in the table
    output_table.loc[ID, "minO2"] = min_o2_to_add
    output_table.loc[ID, "dH"] = min_o2_dH

# store output file as csv
output_table.to_csv("final_results.csv")

# this prints nicely in VSCode
output_table

you have to add at least 5.80t so that the carbon content of Fe-Liq drops below 0.1 w% C
you have to add at least 6.20t so that the carbon content of Fe-Liq drops below 0.1 w% C
you have to add at least 5.40t so that the carbon content of Fe-Liq drops below 0.1 w% C
you have to add at least 5.60t so that the carbon content of Fe-Liq drops below 0.1 w% C
you have to add at least 6.00t so that the carbon content of Fe-Liq drops below 0.1 w% C


,mass,T,Fe,C,Si,Mn,P,S,minO2,dH
ID,,,,,,,,,,
0,97.09,1306.1,94.88,4.01,0.54,0.43,0.109,0.019,5.8,-2.338688e+10
1,99.12,1361.6,94.71,4.28,0.51,0.39,0.096,0.020,6.2,-2.911878e+10
2,94.30,1357.8,95.02,3.95,0.51,0.40,0.101,0.019,5.4,-2.299501e+10
3,97.41,1340.3,94.96,3.99,0.52,0.41,0.095,0.018,5.6,-2.227887e+10
4,95.73,1349.7,94.69,4.29,0.51,0.40,0.097,0.019,6.0,-2.729102e+10
